# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Capstone Project

Notebook 3: Preprocessing & Modeling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SimpleRNN, GRU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import tensorflow.keras.utils as ku 

In [3]:
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_csv('../data/for_train.csv')

### Sepearting Punctuation

In [5]:
def seperate_punch(text):
    punch = ['...', '.', '[', ']', '(', ')', ';', ':', "'", '/', '"', ',', '?', '*', '!', '-', '$', '%', '&']
    for i in punch:
        text = text.replace(i, ' ' + i + ' ')
    return text

df = df['char_line'].map(seperate_punch)

In [6]:
df.head()

0    jerry :  you know ,  why we ' re here ?  to be out ,  this is out  .  .  .  and out is one of the single most enjoyable experiences of life .  people  .  .  .  did you ever hear people talking about  " we should go out "  ?  this is what they ' re talking about  .  .  .  this whole thing ,  we ' re all out now ,  no one is home .  not one person here is home ,  we ' re all out !  there are people tryin '  to find us ,  they don ' t know where we are .   ( imitates one of these people  " tryin '  to find us "  ;  pretends his hand is a phone )   " did you ring ?  ,  i can ' t find him .  "   ( imitates other person on phone )   " where did he go ?  "   ( the first person again )   " he didn ' t tell me where he was going "  .  he must have gone out .  you wanna go out :  you get ready ,  you pick out the clothes ,  right ?  you take the shower ,  you get all ready ,  get the cash ,  get your friends ,  the car ,  the spot ,  the reservation  .  .  .  there you ' re staring around ,

In [7]:
texts = []
for text in df:
    texts.append(text)

### Tokenizing

In [8]:
maxlen = 200
max_words = 10000
batch_size = 32

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens')

data = pad_sequences(sequences, maxlen=maxlen)
print(f'Shape of data tensor: {data.shape}')

Found 18647 unique tokens
Shape of data tensor: (50639, 200)


In [9]:
sequences[1]

[5,
 703,
 8,
 23,
 15,
 806,
 19,
 18,
 3,
 1062,
 973,
 912,
 174,
 58,
 9,
 1,
 6,
 384,
 3,
 249,
 806,
 2629,
 321,
 170,
 1330,
 3,
 384,
 79,
 38,
 10,
 10,
 1,
 6,
 135,
 554,
 10,
 1,
 6,
 18,
 24,
 113,
 1,
 6,
 2241,
 4,
 79,
 50,
 4,
 432,
 28,
 55,
 377]

In [10]:
data

array([[  16,   46,    1, ...,   38,    7,  192],
       [   0,    0,    0, ...,   28,   55,  377],
       [   0,    0,    0, ...,  220,   17, 1954],
       ...,
       [   0,    0,    0, ...,   17,   30, 1921],
       [   0,    0,    0, ...,   17,   55,  219],
       [   0,    0,    0, ...,   64,   89,   21]], dtype=int32)

In [11]:
texts[1]

"jerry :  seems to me ,  that button is in the worst possible spot .   ( talking about george ' s shirt )  the second button literally makes or breaks the shirt ,  look at it :  it ' s too high !  it ' s in no - man ' s - land ,  you look like you live with your mother . "

### Model 1 Simple Modeling - Test Water
https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms
& Francoise Chollet

In [13]:
def get_training_data(data, maxlen):
    
    X, y = data[:,:-1], data[:,-1]
    y = ku.to_categorical(y, num_classes=max_words)
    
    return X, y

In [14]:
X, y = get_training_data(data, maxlen)

In [15]:
X.shape

(50639, 199)

In [45]:
model = Sequential()

model.add(Embedding(max_words, output_dim=maxlen))
model.add(SimpleRNN(32))
model.add(Dense(max_words, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

history = model.fit(X, y,
                   epochs=100,
                   verbose=1)

Epoch 1/100
1603/1603 [==============================] - 217s 135ms/step - loss: 7.0584
Epoch 2/100
1603/1603 [==============================] - 206s 129ms/step - loss: 6.6017
Epoch 3/100
1603/1603 [==============================] - 208s 130ms/step - loss: 6.4238
Epoch 4/100
1603/1603 [==============================] - 207s 129ms/step - loss: 6.3002
Epoch 5/100
1603/1603 [==============================] - 210s 131ms/step - loss: 6.2068
Epoch 6/100
1603/1603 [==============================] - 209s 131ms/step - loss: 6.1631
Epoch 7/100
1603/1603 [==============================] - 217s 135ms/step - loss: 6.5028
Epoch 8/100
1603/1603 [==============================] - 212s 132ms/step - loss: 6.7308
Epoch 9/100
1603/1603 [==============================] - 208s 129ms/step - loss: 6.8238
Epoch 10/100
1603/1603 [==============================] - 206s 128ms/step - loss: 6.9056
Epoch 11/100
1603/1603 [==============================] - 211s 131ms/step - loss: 6.9744
Epoch 12/100
1603/1603 [======

In [36]:
def generate_text(seed_text, next_words, model, maxlen):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=maxlen-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text

In [46]:
generate_text('soup nazi', 100, model, 400)

"soup nazi do it is it up there now ' s you doing here now ' s him huh huh huh right now ' s you doing here again now jerry yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah"

For this model I used keras tokenizer to tokenize the data, and I used SimpleRNN to test out: the result is 

### Model 2 - character based's way with generator
https://github.com/TannerGilbert/Tutorials/blob/master/Keras-Tutorials/4.%20LSTM%20Text%20Generation/Keras%20LSTM%20Text%20Generation.ipynb
https://www.kaggle.com/valkling/pythonicpythonscript4making-seinfeld-scripts
char RNN by karparchy

### Model 3 - TEXTRNN

### Model 4 - GPT-2

shared by Hilary
https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce#scrollTo=LdpZQXknFNY3

Script Buddy V2
https://github.com/cdpierse/script_buddy_v2/blob/master/script_buddy/script_generation.ipynb

Teach GPT-2 sense of humor
https://github.com/mf1024/Transformers/blob/master/Teaching%20GPT-2%20a%20sense%20of%20humor.ipynb